In [16]:
%matplotlib inline

In [17]:
import matplotlib.pyplot as plt
import math
import numpy as np
import scipy as sp
np.seterr(divide='ignore', invalid='ignore')

from pytpc.hdfdata import HDFDataFile

import pytpc
from pytpc.tpcplot import pad_plot, chamber_plot
from pytpc.constants import degrees, pi

import csv
import yaml
import h5py

# Keeping charge data during discretization - sparse matrix implementation

In [18]:
evt_id=107

In [33]:
with pytpc.HDFDataFile('/home/taylor/Documents/independent-research/event-gen/data/ptrain_10000.h5', 'r') as f:
    n_evts = len(f)
    evt = f[evt_id]
    t = evt.traces['data']
    new_xyz = evt.xyzs(peaks_only=True, return_pads=True, baseline_correction=False, cg_times=False)
    print(len(new_xyz))

696


In [20]:
DETECTOR_LENGTH = 1000.0
DETECTOR_RADIUS = 275.0
x_disc = 20
y_disc = 20
z_disc = 20

In [70]:
#NO CHARGE
discElements = x_disc*y_disc*z_disc

buckets = []

num_pts = 0
total_charge = 0

for point in new_xyz:
    x_bucket = math.floor(((point[0]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*x_disc)
    y_bucket = math.floor(((point[1]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*y_disc)
    z_bucket = math.floor((point[2]/DETECTOR_LENGTH)*z_disc)

    bucket_num = z_bucket*x_disc*y_disc + x_bucket + y_bucket*x_disc
    
    #sparse matrix implementation
    buckets.append(bucket_num)
    
    num_pts += 1
    total_charge += point[3]

#create csr matrix where data is np.ones(len(col)) and row np.zeros(len(col)) of size (1, discElements)
cols = np.unique(buckets)
rows = np.zeros(len(cols))
data = np.ones(len(cols))
discretized_data_sparse  = sp.sparse.csr_matrix((data, (rows, cols)), shape=(1, discElements))

print("buckets filled: " + str(discretized_data.sum()))
print("Total charge: " + str(total_charge))

buckets filled: 70.0
Total charge: 266397.0


In [73]:
print(discretized_data_sparse)

  (0, 210)	1.0
  (0, 631)	1.0
  (0, 632)	1.0
  (0, 650)	1.0
  (0, 651)	1.0
  (0, 652)	1.0
  (0, 653)	1.0
  (0, 654)	1.0
  (0, 669)	1.0
  (0, 670)	1.0
  (0, 674)	1.0
  (0, 689)	1.0
  (0, 690)	1.0
  (0, 694)	1.0
  (0, 710)	1.0
  (0, 711)	1.0
  (0, 714)	1.0
  (0, 733)	1.0
  (0, 734)	1.0
  (0, 752)	1.0
  (0, 753)	1.0
  (0, 1010)	1.0
  (0, 1011)	1.0
  (0, 1012)	1.0
  (0, 1013)	1.0
  :	:
  (0, 1428)	1.0
  (0, 1447)	1.0
  (0, 1448)	1.0
  (0, 1467)	1.0
  (0, 1487)	1.0
  (0, 1495)	1.0
  (0, 1507)	1.0
  (0, 1515)	1.0
  (0, 1527)	1.0
  (0, 1528)	1.0
  (0, 1534)	1.0
  (0, 1535)	1.0
  (0, 1548)	1.0
  (0, 1549)	1.0
  (0, 1553)	1.0
  (0, 1554)	1.0
  (0, 1569)	1.0
  (0, 1570)	1.0
  (0, 1571)	1.0
  (0, 1572)	1.0
  (0, 1573)	1.0
  (0, 1808)	1.0
  (0, 1809)	1.0
  (0, 1810)	1.0
  (0, 1828)	1.0


In [75]:
discElements = x_disc*y_disc*z_disc


buckets = []
charges = []
num_pts = 0

for point in new_xyz:
    x_bucket = math.floor(((point[0]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*x_disc)
    y_bucket = math.floor(((point[1]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*y_disc)
    z_bucket = math.floor((point[2]/DETECTOR_LENGTH)*z_disc)

    bucket_num = z_bucket*x_disc*y_disc + x_bucket + y_bucket*x_disc
    
    #sparse matrix implementation
    buckets.append(bucket_num)
    charges.append(point[3])
    
    num_pts += 1

#create csr matrix where data is chrage and row np.zeros(len(col)) of size (1, discElements)
cols = buckets
rows = np.zeros(len(cols))
data = charges

#automatically sums data entries for data occuring at the same point
#no need for sum_duplicates()
discretized_data_sparse_CHARGE  = sp.sparse.csr_matrix((data, (rows, cols)), shape=(1, discElements))


print("buckets filled: " + str(len(np.unique(buckets))))
print("charge acumulated: " + str(sum(data)))

buckets filled: 70
charge acumulated: 266397.0


In [74]:
print(discretized_data_sparse_CHARGE)

  (0, 210)	0.0
  (0, 631)	2268.0
  (0, 632)	2014.0
  (0, 650)	12113.0
  (0, 651)	6619.0
  (0, 652)	5714.0
  (0, 653)	8884.0
  (0, 654)	1048.0
  (0, 669)	6893.0
  (0, 670)	5243.0
  (0, 674)	7394.0
  (0, 689)	5529.0
  (0, 690)	9585.0
  (0, 694)	6234.0
  (0, 710)	19520.0
  (0, 711)	4932.0
  (0, 714)	7366.0
  (0, 733)	5725.0
  (0, 734)	2159.0
  (0, 752)	3345.0
  (0, 753)	2258.0
  (0, 1010)	2713.0
  (0, 1011)	4297.0
  (0, 1012)	4034.0
  (0, 1013)	1620.0
  :	:
  (0, 1428)	3036.0
  (0, 1447)	2921.0
  (0, 1448)	243.0
  (0, 1467)	3340.0
  (0, 1487)	2940.0
  (0, 1495)	3630.0
  (0, 1507)	3775.0
  (0, 1515)	4742.0
  (0, 1527)	1714.0
  (0, 1528)	2301.0
  (0, 1534)	2441.0
  (0, 1535)	1514.0
  (0, 1548)	3302.0
  (0, 1549)	2381.0
  (0, 1553)	2366.0
  (0, 1554)	3649.0
  (0, 1569)	1068.0
  (0, 1570)	3960.0
  (0, 1571)	3997.0
  (0, 1572)	3481.0
  (0, 1573)	2167.0
  (0, 1808)	435.0
  (0, 1809)	3808.0
  (0, 1810)	140.0
  (0, 1828)	830.0


In [79]:
#testing sparse matrix automatic duplicate summing
row = [0, 0, 1, 2, 3]
col = [0, 0, 1, 2, 3]
data = [1, 100, 1, 1, 1]


sp.sparse.csr_matrix( (data, (row, col)), shape =(4,4) ).todense()

matrix([[101,   0,   0,   0],
        [  0,   1,   0,   0],
        [  0,   0,   1,   0],
        [  0,   0,   0,   1]], dtype=int64)

In [80]:
def discretizeGridCharge(xyz, x_disc, y_disc, z_disc):
    """Discretizes AT-TPC point cloud data using a grid geometry by totalling
    charge of hits in a given rectangular bucket.

    Parameters
    ----------
    xyz    : point cloud data with shape (n,5) where n is the number of traces
    x_disc : number of slices in x
    y disc : number of slices in y
    z_disc : number of slices in z

    Returns
    -------
    The discretized data in a csr sparse matrix of shape (1, x_disc*y_disc*z_disc)
    """

    #calculate desired discretization resolution
    discElements = x_disc*y_disc*z_disc

    #calculate dimensional increments
    x_inc = (2*DETECTOR_RADIUS)/x_disc
    y_inc = (2*DETECTOR_RADIUS)/y_disc
    z_inc = DETECTOR_LENGTH/z_disc

    buckets = []
    charges = []

    for point in new_xyz:
        x_bucket = math.floor(((point[0]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*x_disc)
        y_bucket = math.floor(((point[1]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*y_disc)
        z_bucket = math.floor((point[2]/DETECTOR_LENGTH)*z_disc)

        bucket_num = z_bucket*x_disc*y_disc + x_bucket + y_bucket*x_disc
        buckets.append(bucket_num)
        charges.append(point[3])


    cols = buckets
    rows = np.zeros(len(cols))
    data = charges

    #automatically sums charge values for data occuring at the (row, col)
    discretized_data_sparse_CHARGE  = sp.sparse.csr_matrix((data, (rows, cols)), shape=(1, discElements))
    return discretized_data_sparse_CHARGE

In [82]:
trial = discretizeGridCharge(new_xyz, x_disc, y_disc, z_disc)